In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
text = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
text

In [ ]:
import pandas as pd
import numpy as np
from keras import regularizers, optimizers
from keras.layers.experimental.preprocessing import TextVectorization
from keras.layers import Embedding, Dense, Dropout, Input, LSTM, GlobalMaxPool1D
from keras.models import Sequential
from keras.initializers import Constant
import tensorflow as tf
import spacy

# download and import the large english model.
#!python -m spacy download en_core_web_lg
#import en_core_web_lg

nlp = spacy.load('en_core_web_lg')
Vectorizer = TextVectorization()

#load the data 
text = pd.read_csv('../input/commonlitreadabilityprize/train.csv')

#fit the vectorizer on the text and extract the corpus vocabulary
Vectorizer.adapt(text.excerpt.to_numpy())
vocab = Vectorizer.get_vocabulary()

#generate the embedding matrix
num_tokens = len(vocab)
embedding_dim = len(nlp('The').vector)
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for i, word in enumerate(vocab):
    embedding_matrix[i] = nlp(word).vector

#Load the embedding matrix as the weights matrix for the embedding layer and set trainable to False
Embedding_layer=Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False)

#build the model.  This is a bigger one, but it works well on this problem.


#fit the model


In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5,verbose=1,monitor='val_loss'),
    #tf.keras.callbacks.ModelCheckpoint(filepath='Read_model',verbose=1,monitor='val_loss',save_best_only=True),
    #tf.keras.callbacks.LearningRateScheduler(lr_schedul,verbose=1)
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.9,patience=2,min_lr=1e-30,mode='min',verbose=1,)
    
]

In [ ]:
times = 7

In [ ]:
for i in range(times):
    
    model = Sequential()
    model.add(Input(shape=(1,), dtype=tf.string))
    model.add(Vectorizer)
    model.add(Embedding_layer)
    model.add(LSTM(25, return_sequences=True))
    model.add(GlobalMaxPool1D())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='tanh', 
                    kernel_regularizer = regularizers.l1_l2(l1=1e-5, l2=1e-4)))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='tanh', 
                    kernel_regularizer = regularizers.l1_l2(l1=1e-5, l2=1e-4)))    
    model.add(Dense(1))

    adam = optimizers.Adam(learning_rate=.01)
    model.compile(optimizer = adam, loss = 'mean_squared_error', metrics = None)
    
    
    model.fit(text.excerpt,
          text.target,
          batch_size = 500,
          epochs = 80,
          callbacks=my_callbacks,
          validation_split=.2)
    
    model.save('Read_model_'+ str(i))

#print(model.summary())

In [ ]:
#adam = optimizers.Adam(learning_rate=.001, decay=1e-2)
#model.compile(optimizer = adam, loss = 'mean_squared_error', metrics = None)

In [ ]:
#model1 = keras.models.load_model('./Read_model')

In [ ]:
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
pred = []

for i in range(times):
    
    model1 = keras.models.load_model('./Read_model_' + str(i))
    a = model1.predict(test.excerpt)
    pred.append(a)

In [ ]:
np.mean(pred,axis=0)

In [ ]:
test['target'] = np.mean(pred,axis=0)

test[['id','target']].to_csv('submission.csv',index = False)